In [33]:
import requests
import pandas as pd

In [36]:
import requests
import pandas as pd

# 1. Classement partiel (équipes avec des stats)
url_table = "https://www.thesportsdb.com/api/v1/json/3/lookuptable.php?l=4334-french-ligue-1&s=2025-2026"
r = requests.get(url_table).json()
table = r.get("table", [])

data = pd.DataFrame([{
    "Team": team["strTeam"],
    "PTS": int(team["intPoints"]),
    "Diff": int(team["intGoalDifference"]),
    "Played": int(team["intPlayed"])
} for team in table])

# 2. Liste complète des clubs
url_teams = "https://www.thesportsdb.com/api/v1/json/3/lookup_all_teams.php?id=4334"
teams = requests.get(url_teams).json()["teams"]
all_teams = pd.DataFrame([{"Team": t["strTeam"]} for t in teams])

# 3. Fusion
full_data = all_teams.merge(data, on="Team", how="left").fillna(0)

# 4. Conversion en int
full_data["PTS"] = full_data["PTS"].astype(int)
full_data["Diff"] = full_data["Diff"].astype(int)
full_data["Played"] = full_data["Played"].astype(int)

# 5. Classement par points puis diff
full_data = full_data.sort_values(by=["PTS", "Diff"], ascending=False).reset_index(drop=True)

# 6. Ajouter la position
full_data.insert(0, "Pos", full_data.index + 1)

# Affichage
print("🏆 Classement Ligue 1 – 2025/2026 (toutes équipes) :")
display(full_data)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [35]:
# Construction du DataFrame
data = pd.DataFrame([{
    "Pos": team["intRank"],
    "Team": team["strTeam"],
    "M": team["intPlayed"],
    "W": team["intWin"],
    "D": team["intDraw"],
    "L": team["intLoss"],
    "G": team["intGoalsFor"],
    "GA": team["intGoalsAgainst"],
    "Diff": team["intGoalDifference"],
    "PTS": team["intPoints"]
} for team in table])

🏆 Classement Ligue 1 – 2025/2026 (toutes équipes) :


,Pos,Team,PTS,Diff,Played
0,1,Bolton Wanderers,0,0,0
1,2,Wigan Athletic,0,0,0
2,3,Blackpool,0,0,0
3,4,Doncaster Rovers,0,0,0
4,5,Barnsley,0,0,0
5,6,Peterborough United,0,0,0
6,7,Reading,0,0,0
7,8,Cardiff City,0,0,0
8,9,Plymouth Argyle,0,0,0
9,10,Luton Town,0,0,0


In [24]:
def get_team_info(team_name, data):
    team = data[data["Team"] == team_name]
    if team.empty:
        raise ValueError(f"⚠️ L'équipe '{team_name}' n'existe pas dans le classement. "
                         f"Noms disponibles : {list(data['Team'].unique())}")
    return team.iloc[0]

def generate_odds(teamA_name, teamB_name, data):
    """
    Génère les cotes 1X2 pour un match entre teamA et teamB.
    
    teamA_name, teamB_name : noms des équipes (chaînes)
    data : DataFrame du classement (doit contenir 'Team', 'PTS', 'Diff')
    """
    # Extraction des infos depuis le DataFrame
    teamA = get_team_info(teamA_name, data)
    teamB = get_team_info(teamB_name, data)

    # Score de force basé sur les points et la différence de buts
    strengthA = int(teamA["PTS"]) + int(teamA["Diff"]) * 0.3
    strengthB = int(teamB["PTS"]) + int(teamB["Diff"]) * 0.3

    # Probabilités brutes
    pA = strengthA / (strengthA + strengthB)
    pB = strengthB / (strengthA + strengthB)
    pDraw = 0.15 + 0.1 * (1 - abs(pA - pB))  # nul plus probable si forces proches

    # Normalisation
    total = pA + pB + pDraw
    pA /= total
    pB /= total
    pDraw /= total

    # Conversion en cotes décimales (1/probabilité) + marge bookmaker
    margin = 1.05
    oddsA = round((1 / pA) * margin, 2)
    oddsDraw = round((1 / pDraw) * margin, 2)
    oddsB = round((1 / pB) * margin, 2)

    return {
        "Match": f"{teamA_name} vs {teamB_name}",
        "1": oddsA,
        "X": oddsDraw,
        "2": oddsB
    }

In [25]:
season_data = pd.DataFrame([{
    "Team": team["strTeam"],
    "M": int(team["intPlayed"]),
    "W": int(team["intWin"]),
    "D": int(team["intDraw"]),
    "L": int(team["intLoss"]),
    "G": int(team["intGoalsFor"]),
    "GA": int(team["intGoalsAgainst"]),
    "Diff": int(team["intGoalDifference"]),
    "PTS": int(team["intPoints"])
} for team in table])
try:
    home_team = "Rennes"
    away_team = "Lyon"

    odds = generate_odds(home_team, away_team, season_data)

    print(f"\n📊 Predicted Odds: {home_team} vs {away_team}")
    for k, v in odds.items():
        print(f"  {k}: {v}")

except Exception as e:
    print("Error:", e)

Error: ⚠️ L'équipe 'Rennes' n'existe pas dans le classement. Noms disponibles : ['Paris SG', 'Lyon', 'Lille', 'Monaco', 'Lens']
